In [ ]:
import pandas as pd
import numpy as np
import os
from astropy.utils.data import download_file
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
import warnings
import shutil
from astropy.io import fits
import astropy.coordinates as coord
import astropy.units as u
from astropy.io import ascii
from astropy.coordinates import SkyCoord
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.preprocessing import Normalizer, StandardScaler, MinMaxScaler, MaxAbsScaler, RobustScaler
import matplotlib.pyplot as plt
import seaborn as sns
#from playsound import playsound
import requests
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, MultiChoice, CustomJS, ColumnDataSource, LinearColorMapper, CategoricalColorMapper, ColorBar
from bokeh.layouts import column
from bokeh.palettes import Plasma256
plt.rcParams["axes.prop_cycle"] = plt.cycler("color", plt.cm.Dark2.colors)
output_notebook()

In [ ]:
bokeh.plot()

In [ ]:
hardness_data_swift = pd.read_pickle('DataFrames/hardness_data.dat')
display(hardness_data_swift)
print(hardness_data_swift.loc['GRB140716A-1'])

In [ ]:
fluence_data_swift = pd.read_pickle('DataFrames/fluence_data.dat')
display(fluence_data_swift)

In [ ]:
fluence_data_BATSE = pd.read_pickle('DataFrames/fluence_data_BATSE.dat')
display(fluence_data_BATSE)

In [ ]:
pd.read_pickle("DataFrames/duration_data_BATSE.dat")

In [ ]:
pd.read_pickle("DataFrames/hardness_data_BATSE.dat")

In [ ]:
pd.read_pickle("DataFrames/peak_flux_BATSE.dat")

In [ ]:
pd.read_pickle("DataFrames/duration_data.dat").loc['GRB061210']

In [ ]:
pd.read_pickle("DataFrames/duration_data.dat").loc['GRB090531B']

In [ ]:
pd.read_csv('DataFrames/fluence_data_Fermi.csv')

In [ ]:
pd.read_csv('DataFrames/duration_data_Fermi.csv', index_col=0).loc['GRB200826187']

In [ ]:
lc_swift = pd.read_csv("LightCurves/GRB041217_lc.dat", sep = ' ', header = None)
lc_swift

In [ ]:
file = fits.open('Fermi/bn080714086.fit')

In [ ]:
file.info()

In [ ]:
file[0].header

In [ ]:
file[2].header

In [ ]:
file[2].data.field('PARAM1')

In [ ]:
data = file[2]

In [ ]:
data.header

In [ ]:
pd.DataFrame(data.data.field('NRGFLNC')).loc[:,0].sum()/417

In [ ]:
data.data.field('TIMEBIN')

In [ ]:
full = fits.open('Fermi/bn080714086.fit')[2].data.field('NRGFLUX')
full

In [ ]:
df = pd.DataFrame(fits.open('Fermi/bn080724401.fit')[2].data.field('NRGFLUX'))
df

In [ ]:
fluence_data = pd.read_csv('DataFrames/fluence_data_Fermi.csv', index_col=0)
fluence_data.loc['GRB200826187','fluence'] = '4.8E-06'

def normalize_lc(filename): # Normalize by fluence
    grbname = filename[8:-4]
    #Cut lightcurve
    lc = pd.DataFrame(fits.open(filename)[2].data.field('NRGFLUX'))
    lc = lc.squeeze() / float(fluence_data.loc[f'GRB{grbname}','fluence'])
    return len(lc), lc # Return length and the normalized lightcurve

In [ ]:
plt.plot(normalize_lc('Fermi/bn090510016.fit')[1])

In [ ]:
print(fluence_data.loc['GRB180201780','fluence'])

In [ ]:
plt.plot(normalize_lc('Fermi/bn080727964.fit')[1])

In [ ]:
normalize_lc('Fermi/bn080727964.fit')
print(fluence_data.loc['GRB080727964','fluence'])

In [ ]:
fits.open('Fermi/bn080727964.fit')[2].data.field('NRGFLUX')[215:217]

In [ ]:
#duration = pd.read_pickle('DataFrames/duration_data.dat')
#duration.loc['GRB211211A'] = ['1088940', 2.932, 53.256, np.NaN, np.NaN, 50.324]
#duration = duration.sort_values('GRBname', ascending=False)
#duration.to_pickle("DataFrames/duration_data.dat")
#display(duration)

In [ ]:
#fluence = pd.read_pickle('DataFrames/fluence_data.dat').to_frame().reset_index()
#fluence.loc[len(fluence.index)] = ['GRB211211A','2.52e-04']
#fluence.columns = ['GRBname', 'fluence']
#fluence = fluence.sort_values('GRBname', ascending=False).set_index('GRBname')
#fluence = fluence.squeeze()
#fluence.to_pickle("DataFrames/fluence_data.dat")
#display(fluence)

In [ ]:
f = fits.open("BATSE/GRB105.fits")
f.info()

In [ ]:
f[0].header

In [ ]:
pd.DataFrame(f[2].data.field('TIMES'))

In [ ]:
df = pd.DataFrame(f[2].data.field('RATES'))
df

In [ ]:
pd.concat([pd.DataFrame(f[2].data.field('TIMES')[:,0]), pd.DataFrame(f[2].data.field('RATES'))], axis = 1)

In [ ]:
duration_data = pd.read_pickle("DataFrames/duration_data_BATSE.dat")
fluence_data = pd.read_pickle("DataFrames/fluence_data_BATSE.dat")
data_problems = pd.read_pickle("DataFrames/data_problems_BATSE.dat")
outliers = pd.read_pickle("DataFrames/outliers_BATSE.dat")


def cut_norm_lc(filename): #Prepare single light curve, cut to T90 and normalize by fluence
    trigger = int(filename[9:-5])
    #Cut lightcurve
    lc = pd.concat([pd.DataFrame(fits.open(filename)[2].data.field('TIMES')[:,0]), pd.DataFrame(fits.open(filename)[2].data.field('RATES'))], axis = 1, ignore_index = True)
    #if str(trigger) in data_problems.index:
        #lc = lc.drop(index=data_problems.loc[str(trigger),'location'])
    if str(trigger) in list(outliers):
        return None
    lc = lc.loc[lc.loc[:,0].apply(lambda x: duration_data.loc[trigger,'start_T90'] <= x and x <= (duration_data.loc[trigger,'start_T90'] + duration_data.loc[trigger,'T90']))]
    lc.reset_index(drop=True,inplace=True)
    lc = lc.iloc[:,[1,2,3,4]] / float(fluence_data.loc[trigger, 'fluence'])
    return len(lc), lc # Return length and the cut lightcurve

cut_norm_lc('BATSE/GRB5458.fits')

In [ ]:
pd.DataFrame(fits.open("BATSE/GRB5458.fits")[2].data.field('RATES'))

In [ ]:
#lc = pd.DataFrame(fits.open('BATSE/GRB5992.fits')[2].data.field('RATES').byteswap().newbyteorder())[80:90]
fluence_data = pd.read_pickle("DataFrames/fluence_data_BATSE.dat")
duration_data = pd.read_pickle("DataFrames/duration_data_BATSE.dat")
data_problems = pd.read_pickle("DataFrames/data_problems_BATSE.dat")
for trigger in list(data_problems.index):
    filename = f'BATSE/GRB{trigger}.fits'
    trigger = int(trigger)
    print(trigger)
    display(duration_data.loc[trigger])
    lc = pd.concat([pd.DataFrame(fits.open(filename)[2].data.field('TIMES')), pd.DataFrame(fits.open(filename)[2].data.field('RATES'))], axis = 1, ignore_index = True)
    lc.loc[data_problems.loc[str(trigger),'location']] /= 100
    lc = lc.loc[lc.loc[:,0].apply(lambda x: duration_data.loc[trigger,'start_T90'] <= x and x <= (duration_data.loc[trigger,'start_T90'] + duration_data.loc[trigger,'T90']))]
    lc.iloc[:,0] = pd.to_timedelta(pd.Series(lc.iloc[:,0]), unit = 's')
    lc.iloc[:,1] = pd.to_timedelta(pd.Series(lc.iloc[:,1]), unit = 's')
    lc = lc.resample('64ms', on=0).mean().bfill()
    lc.reset_index(drop=True,inplace=True)
    lc = lc.loc[:,[2,3,4,5]]
    display(lc)
    plt.plot(lc)
    plt.show()

In [ ]:
fluence_data = pd.read_pickle("DataFrames/fluence_data_BATSE.dat")
duration_data = pd.read_pickle("DataFrames/duration_data_BATSE.dat")
data_problems = pd.read_pickle("DataFrames/data_problems_BATSE.dat")

def cut_norm_lc(filename): #Prepare single light curve, cut to T90 and normalize by fluence
    trigger = int(filename[9:-5])
    #Cut lightcurve
    lc = pd.concat([pd.DataFrame(fits.open(filename)[2].data.field('TIMES')), pd.DataFrame(fits.open(filename)[2].data.field('RATES'))], axis = 1, ignore_index = True)
    if str(trigger) in data_problems.index:
        lc.loc[data_problems.loc[str(trigger),'location']] /= 100
    #if f'GRB{str(trigger)}' in list(outliers):
        #lc = lc.drop(index=data_problems.loc[str(trigger),'location'])
    lc = lc.loc[lc.loc[:,0].apply(lambda x: duration_data.loc[trigger,'start_T90'] <= x and x <= (duration_data.loc[trigger,'start_T90'] + duration_data.loc[trigger,'T90']))]
    #lc = lc[lc[1] - lc[0] <= 0.006]
    lc.reset_index(drop=True,inplace=True)
    #lc = lc.iloc[:,[1,2,3,4]] / float(fluence_data.loc[trigger,'fluence'])
    lc.iloc[:,0] = pd.to_timedelta(pd.Series(lc.iloc[:,0]), unit = 's')
    lc.iloc[:,1] = pd.to_timedelta(pd.Series(lc.iloc[:,1]), unit = 's')
    print(lc.iloc[:,1] - lc.iloc[:,0])
    lc = lc.resample('64ms', on=0).mean().pad()
    lc.reset_index(drop=True,inplace=True)
    lc = lc.loc[:,[2,3,4,5]] / float(fluence_data.loc[trigger,'fluence'])
    return len(lc), lc # Return length and the cut lightcurve
trig = 8121
print(duration_data.loc[trig,'T90']/0.064)
cut_norm_lc(f'BATSE/GRB{trig}.fits')
plt.plot(cut_norm_lc(f'BATSE/GRB{trig}.fits')[1])

In [ ]:
data = pd.DataFrame(f[2].data.field('TIMES').byteswap().newbyteorder())
trigger = 1192
data = data.loc[data.loc[:,0].apply(lambda x: duration_data.loc[trigger,'start_T90'] <= x and x <= (duration_data.loc[trigger,'start_T90'] + duration_data.loc[trigger,'T90']))]
#data = data.loc[data.loc[:,0].apply(lambda x: duration_data.loc[trigger,'start_T90'] <= x and x <= (duration_data.loc[trigger,'start_T90'] + duration_data.loc[trigger,'T90']))]
data = data[data[1] - data[0] <= 0.005]
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    #print(data)
    print(data[1] - data[0])

In [ ]:
filename = "BATSE/GRB6137.fits"
lc = pd.concat([pd.DataFrame(fits.open(filename)[2].data.field('TIMES')[:,0]), pd.DataFrame(fits.open(filename)[2].data.field('RATES'))], axis = 1, ignore_index = True)
lc

In [ ]:
path = "BATSE/"
trig = []
loc = []
'''
for file in os.listdir(path):
    df = pd.DataFrame(fits.open(path + file)[2].data.field('RATES').byteswap().newbyteorder())
    if (df>50000).any(1).any():
        index = list(df.loc[(df>50000).any(1)].index)
        check_outlier = (df.iloc[index[0]][0] > 10 * df.iloc[index[0]-1][0]) and (df.iloc[index[0]][0] > 10 * df.iloc[index[0]+1][0])
        if check_outlier:
            print(file)
            print(index)
            i = index[0]
            print(df[i-2:i+3])
            if len(index) < 5:
                trig.append(file[3:-5])
                loc.append(index)
'''

In [ ]:
len(trig)

In [ ]:
data_problems = pd.DataFrame(data={'Trigger':trig,'location':loc}).set_index('Trigger')
display(data_problems)
#data_problems.to_pickle("DataFrames/data_problems_BATSE.dat")
print(data_problems.index)

In [ ]:
filename = 'BATSE/GRB1453.fits'
trigger = 1453
lc = pd.concat([pd.DataFrame(fits.open(filename)[2].data.field('TIMES')[:,0]), pd.DataFrame(fits.open(filename)[2].data.field('RATES'))], axis = 1, ignore_index = True)
if str(trigger) in data_problems.index:
    lc = lc.drop(index=data_problems.loc[str(trigger),'location'])
    display(lc[80:])

In [ ]:
old = pd.read_table('Classifications.txt', sep = '\s+', skiprows = 14, names = ['GRBname', 'fluence', 'type']).set_index('GRBname')
#old[]
# 1254 total bursts included
old[(old == 'Discarded').any(axis=1)]

In [ ]:
new = pd.read_pickle('DataFrames/classification.dat')
new

In [ ]:
union = pd.concat([old.loc[:, 'type'], new], axis=1, ignore_index=True)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(union[union[0] != union[1]].dropna())

#GRB180418A
#GRB121226A
#GRB050724

In [ ]:
cat1 = pd.read_csv('DataFrames/classification_BATSE.txt', index_col=0)
cat2 = pd.read_csv('DataFrames/classification_BATSE_UMAP.txt', index_col=0)
union = pd.concat([cat1.loc[:, 'type'], cat2], axis=1, ignore_index=True)
miscat = union[union[0] != union[1]]
miscat.columns = ['t-SNE', 'UMAP']
len(miscat.index)
miscat.groupby(miscat.UMAP).get_group('S').size/2
#miscat.to_csv('DataFrames/mismatching_bursts_BATSE.txt')

In [ ]:
cat1 = pd.read_csv('DataFrames/classification_BATSE_UMAP.txt', index_col=0)
cat1.groupby(cat1.type).get_group('S')

In [ ]:
duration_data = pd.read_pickle('DataFrames/duration_data.dat')
fluence_data = pd.read_pickle('DataFrames/fluence_data.dat')


def cut_norm_lc(filename): #Prepare single light curve, cut to T100 and normalize by fluence
    grbname = filename[12:-7]
    #Cut lightcurve
    lc = pd.read_csv(filename, sep = ' ', header = None)
    lc = lc.loc[:, [0, 1, 3, 5, 7]]
    lc = lc.loc[lc.loc[:,0].apply(lambda x: duration_data.T100_start[grbname] <= x and x <= duration_data.T100_end[grbname])]
    lc.reset_index(drop=True,inplace=True)
    lc = lc.iloc[:,[1,2,3,4]] / float(fluence_data.loc[grbname, 'fluence'])
    return len(lc), lc # Return length and the cut lightcurve

print(cut_norm_lc('LightCurves/GRB041219B_lc.dat'))
print(duration_data.loc['GRB041219B'])

In [ ]:
plt.plot(cut_norm_lc('LightCurves/GRB041219B_lc.dat')[1])

In [ ]:
plt.plot(cut_norm_lc('LightCurves/GRB050724_lc.dat')[1])
cut_norm_lc('LightCurves/GRB050724_lc.dat')[0]
duration_data.loc['GRB041219B']

In [ ]:
#problem_files = list(pd.read_table('Error_log.txt', on_bad_lines = 'skip', header = None)[0].str.strip())
#for file in problem_files:
    #lc = cut_norm_lc(f'LightCurves/{file}_lc.dat')
    #print(file)
    #print(lc[0])
    #plt.plot(lc[1])
    #plt.show()

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(pd.read_pickle('DataFrames/outliers_BATSE.dat'))
print('')

In [ ]:
#for grb in list(pd.Series(pd.read_pickle('DataFrames/outliers_BATSE.dat'))):
    #print(grb)
    #plt.plot(cut_norm_lc(f'BATSE/{grb}.fits')[1])
    #plt.show()

In [ ]:
duration = pd.read_pickle("DataFrames/duration_data.dat").iloc[:,[1,2,3,4]]
fluence = pd.read_pickle('DataFrames/fluence_data.dat')
display(fluence)
hardness = pd.read_pickle('DataFrames/hardness_data.dat')
summary = pd.concat([duration, fluence, hardness], axis=1)
#display(summary)
#summary.to_csv('summary_Swift.csv')
print(summary.T100_start)

In [ ]:
def cut_norm_lc(filename): #Prepare single light curve, cut to T100 and normalize by fluence
    grbname = filename[12:-7]
    #Cut lightcurve
    lc = pd.read_csv(filename, sep = ' ', header = None)
    lc = lc.loc[:, [0, 1, 3, 5, 7]]
    lc = lc.loc[lc.loc[:,0].apply(lambda x: summary.T100_start[grbname] <= x and x <= summary.T100_end[grbname])]
    lc.reset_index(drop=True,inplace=True)
    lc = lc.iloc[:,[1,2,3,4]] / float(summary.fluence[grbname])
    return len(lc), lc # Return length and the cut lightcurve
plt.plot(cut_norm_lc('LightCurves/GRB121226A_lc.dat')[1])

In [ ]:
summary = pd.read_csv('summary_Swift.csv', index_col=0)#.set_index('GRBname')
#summary.to_csv('summary_Swift.csv')
display(summary)
def cut_norm_lc(filename): #Prepare single light curve, cut to T100 and normalize by fluence
    grbname = filename[12:-7]
    #Cut lightcurve
    lc = pd.read_csv(filename, sep = ' ', header = None)
    lc = lc.loc[:, [0, 1, 3, 5, 7]]
    lc = lc.loc[lc.loc[:,0].apply(lambda x: summary.T100_start[grbname] <= x and x <= summary.T100_end[grbname])]
    lc.reset_index(drop=True,inplace=True)
    lc = lc.iloc[:,[1,2,3,4]] / float(summary.fluence[grbname])
    return len(lc), lc # Return length and the cut lightcurve
cut_norm_lc('LightCurves/GRB050724_lc.dat')

In [ ]:
lc_url = f'https://heasarc.gsfc.nasa.gov/FTP/compton/data/batse/trigger/'

In [ ]:
import requests
from bs4 import BeautifulSoup

def get_url_paths(url, ext='', params={}):
    response = requests.get(url, params=params)
    if response.ok:
        response_text = response.text
    else:
        return response.raise_for_status()
    soup = BeautifulSoup(response_text, 'html.parser')
    parent = [url + node.get('href') for node in soup.find_all('a') if '00' in node.get('href')]
    return parent

url = 'https://heasarc.gsfc.nasa.gov/FTP/compton/data/batse/ascii_data/64ms/'
ext = ''
result = get_url_paths(url, ext)
#print(result)
print(len(result))

def get_url_paths2(url, ext='', params={}):
    response = requests.get(url, params=params)
    if response.ok:
        response_text = response.text
    else:
        return response.raise_for_status()
    soup = BeautifulSoup(response_text, 'html.parser')
    parent = [url + node.get('href') for node in soup.find_all('a')]
    return parent

url = 'https://heasarc.gsfc.nasa.gov/FTP/compton/data/batse/ascii_data/64ms/'
ext = ''
result2 = []
for path in result:
    print(get_url_paths2(path,ext))
print()
for path in result:
    print(len(get_url_paths2(path, ext)))
    result2 += get_url_paths2(path, ext)
print()
print(len(result2))

In [ ]:
#bursts = pd.DataFrame(result2)[0].str[74:78]
GRB_Names = pd.read_table("summary/Basic_BATSE_new.dat", sep="\s+", comment='#', header=None, usecols=[0,1,2],index_col=0)
#for burst in bursts:
    #if int(burst) not in GRB_Names.index:
        #print(burst)

In [ ]:
GRB_Names = pd.read_table("summary/Basic_BATSE_new.dat", sep="\s+", comment='#', header=None, usecols=[0,1,2],index_col=0)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(GRB_Names)

In [ ]:
import requests
from bs4 import BeautifulSoup

def get_url_paths(url, params={}):
    response = requests.get(url, params=params)
    if response.ok:
        response_text = response.text
    else:
        return response.raise_for_status()
    soup = BeautifulSoup(response_text, 'html.parser')
    parent = [url + node.get('href') for node in soup.find_all('a') if '20' in node.get('href')]
    return parent

url = 'https://heasarc.gsfc.nasa.gov/FTP/fermi/data/gbm/bursts/'
years = get_url_paths(url)
tally = 0
'''
for year in years:
    print(f'Currently searching through {year}')
    for burst in get_url_paths(year):
        tally += 1
        has_file = False
        for file in get_url_paths(burst+'/current/'):
            if 'bcat_all' in file:
                has_file = True
        if not has_file:
            print(burst)
print(tally)
'''

In [ ]:
def get_url_paths(url, params={}):
    response = requests.get(url, params=params)
    if response.ok:
        response_text = response.text
    else:
        return response.raise_for_status()
    soup = BeautifulSoup(response_text, 'html.parser')
    print(soup)
    parent = [url + node.get('href') for node in soup.find_all('a')]
    return parent

#url = 'https://heasarc.gsfc.nasa.gov/FTP/fermi/data/gbm/triggers/'
#years = get_url_paths(url)

In [ ]:
duration_data = pd.read_csv('DataFrames/duration_data_Fermi.csv', index_col=0)
fluence_data = pd.read_csv('DataFrames/fluence_data_Fermi.csv', index_col=0)
print(len(duration_data))
print(len(fluence_data))

In [ ]:
duration_data = pd.read_csv('DataFrames/duration_data_Fermi.csv', index_col=0)
fluence_data = pd.read_csv('DataFrames/fluence_data_Fermi.csv', index_col=0)


def normalize_lc(filename): # Normalize by fluence
    grbname = filename[8:-4]
    #Cut lightcurve
    lc = pd.DataFrame(fits.open(filename)[2].data.field('NRGFLUX'))
    lc = lc.squeeze() / float(fluence_data.loc[f'GRB{grbname}','fluence'])
    return len(lc), lc # Return length and the normalized lightcurve

In [ ]:
normalize_lc('Fermi/bn081229675.fit')

In [ ]:
pd.read_pickle('DataFrames/flux_data.dat')

In [ ]:
pd.read_pickle('DataFrames/fluence_data.dat')

In [ ]:
duration_data = pd.read_csv('DataFrames/duration_data_Fermi.csv', index_col=0)
grb_names = duration_data.index
bcat_exists = 0
version_numbers = {}
'''
for grb in grb_names:
    for i in range(5):
        if requests.head(f'https://heasarc.gsfc.nasa.gov/FTP/fermi/data/gbm/bursts/20{grb[3:5]}/bn{grb[3:]}/current/glg_bcat_all_bn{grb[3:]}_v0{i}.fit').status_code == 200:
            bcat_exists += 1
            version_numbers[grb] = i
    if bcat_exists % 100 == 0:
        print(f'{bcat_exists} files found')
'''
print(f'{bcat_exists} files total')

In [ ]:
#version_df = pd.DataFrame.from_dict(version_numbers, orient = 'index').reset_index()
#version_df.columns = ['name', 'file version']
#version_df = version_df.set_index('name')
#version_df

In [ ]:
#version_df.to_csv('DataFrames/fermi_file_versions.csv')

In [ ]:
pd.read_csv('DataFrames/fermi_file_versions.csv')

In [ ]:
discarded = pd.read_table('DataFrames/discarded_bursts_Swift.txt')

In [ ]:
discarded

In [ ]:
'GRB190718A_lc.dat' in list(discarded.discarded)

In [ ]:
print(list(discarded.discarded))

In [ ]:
summary.fluence

In [ ]:
pd.read_pickle('DataFrames/fluence_data.dat')

In [ ]:
duration_data = pd.read_pickle('DataFrames/duration_data.dat')
classification = pd.read_csv('DataFrames/classification_Swift.txt')
group = classification.groupby(classification.type)
short = group.get_group('S')
long = group.get_group('L')
plt.hist(np.log10(duration_data.loc[short.GRBname,'T90']), range = (-3,3), bins = 25, color = 'C1', edgecolor = 'C1', alpha = 0.5)
plt.hist(np.log10(duration_data.loc[long.GRBname,'T90']), range = (-3,3), bins = 25, color = 'C4', edgecolor = 'C4', alpha = 0.5)
plt.xlabel('log$T_{90}$', size = 12)

In [ ]:
duration_data = pd.read_csv('DataFrames/duration_data_Fermi.csv').set_index('name')
classification = pd.read_csv('DataFrames/classification_Fermi.txt')
group = classification.groupby(classification.type)
short = group.get_group('S')
long = group.get_group('L')
plt.hist(np.log10(duration_data.loc[short.GRBname,'T90']), range = (-3,3), bins = 25, color = 'C1', edgecolor = 'C1', alpha = 0.5)
plt.hist(np.log10(duration_data.loc[long.GRBname,'T90']), range = (-3,3), bins = 25, color = 'C4', edgecolor = 'C4', alpha = 0.5)
plt.xlabel('log$T_{90}$', size = 12)

In [ ]:
duration_data = pd.read_pickle('DataFrames/duration_data_BATSE.dat')
classification = pd.read_csv('DataFrames/classification_BATSE.txt')
group = classification.groupby(classification.type)
short = group.get_group('S').loc[:,'GRBname']
long = group.get_group('L').loc[:,'GRBname']
plt.hist(np.log10(duration_data.loc[short,'T90']), bins = 25, color = 'C1', edgecolor = 'C1', alpha = 0.5)
plt.hist(np.log10(duration_data.loc[long,'T90']), bins = 25, color = 'C4', edgecolor = 'C4', alpha = 0.5)
plt.xlabel('log$T_{90}$', size = 12)

In [ ]:
duration_data = pd.read_pickle('DataFrames/duration_data_BATSE.dat')
plt.hist(np.log10(duration_data.loc[:5585,'T90']), bins = 20, color = 'C1', edgecolor = 'C1', alpha = 0.5)
plt.xlabel('log$T_{90}$', size = 12)

In [ ]:
duration_data = pd.read_pickle('DataFrames/duration_data_BATSE.dat')

fig, axs = plt.subplots(3,figsize=(8,12),constrained_layout=True)
#figure out how to do logarithmic ticks
plt.xlabel('log$T_{90}$',size=14)
plt.ylabel('Count',size=14)
axs[0].set_title('Current Catalog (2037 bursts)',size=16)
axs[0].hist(np.log10(duration_data.loc[:,'T90']), range = (-2,3), bins = 25, color = 'C7', edgecolor = 'C7', alpha = 0.5)
axs[1].set_title('4B Catalog (1231 bursts)',size=16)
axs[1].hist(np.log10(duration_data.loc[:5585,'T90']), range = (-2,3), bins = 25, color = 'C7', edgecolor = 'C7', alpha = 0.5)
axs[2].set_title('Post-4B Catalog (806 bursts)',size=16)
axs[2].hist(np.log10(duration_data.loc[5589:,'T90']), range = (-2,3), bins = 25, color = 'C7', edgecolor = 'C7', alpha = 0.5)
#plt.savefig('batse_duration_comparison.jpg',dpi=300)

In [ ]:
duration_data = pd.read_pickle('DataFrames/duration_data_BATSE.dat')
classification = pd.read_csv('DataFrames/classification_BATSE_Short.txt')
classification.GRBname = classification.GRBname.str[3:].astype('int')
duration_data = duration_data[duration_data.index.isin(classification.GRBname.tolist())]
display(duration_data)
display(classification)
group = classification.groupby(classification.type)
short = group.get_group('S').loc[:,'GRBname']
long = group.get_group('L').loc[:,'GRBname']
duration_data_s = duration_data.loc[short].reindex_like(duration_data).dropna(subset=['T90'])
duration_data_l = duration_data.loc[long].reindex_like(duration_data).dropna(subset=['T90'])
fig, axs = plt.subplots(3,figsize=(8,12),constrained_layout=True)
#figure out how to do logarithmic ticks
plt.xlabel('log$T_{90}$',size=14)
plt.ylabel('Count',size=14)
axs[0].set_title('Current Catalog (1911 bursts)',size=16)
axs[0].hist(np.log10(duration_data_s.loc[:,'T90']), range = (-2,3), bins = 25, color = 'C1', edgecolor = 'C1', alpha = 0.5)
axs[0].hist(np.log10(duration_data_l.loc[:,'T90']), range = (-2,3), bins = 25, color = 'C2', edgecolor = 'C2', alpha = 0.5)
axs[1].set_title('4B Catalog (1152 bursts)',size=16)
axs[1].hist(np.log10(duration_data_s.loc[:5585,'T90']), range = (-2,3), bins = 25, color = 'C1', edgecolor = 'C1', alpha = 0.5)
axs[1].hist(np.log10(duration_data_l.loc[:5585,'T90']), range = (-2,3), bins = 25, color = 'C2', edgecolor = 'C2', alpha = 0.5)
axs[2].set_title('Post-4B Catalog (759 bursts)',size=16)
axs[2].hist(np.log10(duration_data_s.loc[5589:,'T90']), range = (-2,3), bins = 25, color = 'C1', edgecolor = 'C1', alpha = 0.5)
axs[2].hist(np.log10(duration_data_l.loc[5589:,'T90']), range = (-2,3), bins = 25, color = 'C2', edgecolor = 'C2', alpha = 0.5)
#plt.savefig('batse_duration_comparison_2.jpg',dpi=300)

In [ ]:
duration_data.dropna()

In [ ]:
duration_data_s.size/5+duration_data_l.size/5

In [ ]:
duration_data_s.loc[:5585].size/5+duration_data_l.loc[:5585].size/5

In [ ]:
duration_data_s.loc[5589:].size/5+duration_data_l.loc[5589:].size/5

In [ ]:
duration_data.loc[2464]

In [ ]:
duration_data.loc[:5585].dropna()

In [ ]:
duration_data.loc[5589:].dropna()

In [ ]:
non_fft_dataset = pd.read_pickle('non_fft_dataset_BATSE.dat')
classification = pd.read_csv('DataFrames/classification_BATSE.txt', index_col = 0)
short = classification.groupby(classification.type).get_group('S')
print(short)
#print(short.index)
#print('GRB107' in short.index)#.tolist())
#non_fft_dataset = non_fft_dataset.loc[short.index]
#non_fft_dataset = non_fft_dataset.loc[:, (non_fft_dataset != 0).any(axis=0)]
#non_fft_dataset

In [ ]:
skymap = pd.read_pickle('DataFrames/skymap_data_BATSE.dat')
classification = pd.read_csv('DataFrames/classification_BATSE.txt', index_col=0)
#classification.index = classification.index.str.slice(3).astype(int)
#df = pd.concat([skymap, classification], axis=1).dropna()
#classification.drop(classification.loc[classification.index.isin(df.index)].index)
#df

In [ ]:
data = df.replace({'L': 'C4', 'S': 'C1'})

xarr = np.array(data.iloc[:,0])
yarr = np.array(data.iloc[:,1])
eq = SkyCoord(xarr[:], yarr[:], frame='galactic', unit=u.deg)
gal = eq.galactic

plt.figure(figsize=[10,10])
plt.subplot(111, projection='aitoff')
plt.grid(True)
plt.scatter(gal.l.wrap_at('180d').radian, gal.b.radian, c = data.iloc[:,4])

In [ ]:
file = pd.read_csv('LC/GRB5989.csv', skiprows=1, header=None, sep='\s+')
header = pd.Series(data=file.loc[0].values, index=['trig#', 'npts', 'nlasc', '1preb'], dtype='int64')
display(header)
display(duration_data.loc[5989])
rates = file.drop(0).reset_index(drop=True)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(rates)

In [ ]:
rates[(header['nlasc']-10):(header['nlasc']+5)]

In [ ]:
duration_data = pd.read_pickle("DataFrames/duration_data_BATSE.dat")
fluence_data = pd.read_pickle("DataFrames/fluence_data_BATSE.dat")

def cut_norm_lc(filename): #Prepare single light curve, cut to T90 and normalize by fluence
    trigger = int(filename[6:-4])
    #Cut lightcurve
    file = pd.read_csv(filename, skiprows=1, header=None, sep='\s+')
    header = pd.Series(data=file.loc[0].values, index=['trig#', 'npts', 'nlasc', '1preb'], dtype='int64')
    trig_time = header['nlasc'] + 32
    lc = file.drop(0).reset_index(drop=True)
    lc = lc.loc[(trig_time + duration_data.loc[trigger,'start_T90']/0.064):(trig_time + (duration_data.loc[trigger,'start_T90'] + duration_data.loc[trigger,'T90'])/0.064 - 1)]
    lc.reset_index(drop=True,inplace=True)
    lc = lc.iloc[:,[0,1,2,3]] / float(fluence_data.loc[trigger,'fluence'])
    return len(lc), lc # Return length and the cut lightcurve

filename = 'LC/GRB107.csv'
display(duration_data.loc[int(filename[6:-4])])
display(fluence_data.loc[int(filename[6:-4]), 'fluence'])
length, lc = cut_norm_lc(filename)
print(lc)
plt.plot(lc)

In [ ]:
fluence_data = pd.read_pickle("DataFrames/fluence_data_BATSE.dat")
duration_data = pd.read_pickle("DataFrames/duration_data_BATSE.dat")
data_problems = pd.read_pickle("DataFrames/data_problems_BATSE.dat")

def cut_norm_lc(filename): #Prepare single light curve, cut to T90 and normalize by fluence
    trigger = int(filename[9:-5])
    #Cut lightcurve
    lc = pd.concat([pd.DataFrame(fits.open(filename)[2].data.field('TIMES')), pd.DataFrame(fits.open(filename)[2].data.field('RATES'))], axis = 1, ignore_index = True)
    #if str(trigger) in data_problems.index:
    #    lc.loc[data_problems.loc[str(trigger),'location']] /= 100
    #if f'GRB{str(trigger)}' in list(outliers):
        #lc = lc.drop(index=data_problems.loc[str(trigger),'location'])
    print(lc)
    lc = lc.loc[lc.loc[:,0].apply(lambda x: duration_data.loc[trigger,'start_T90'] - 0.064 < x and x <= (duration_data.loc[trigger,'start_T90'] + duration_data.loc[trigger,'T90']))]
    #lc = lc[lc[1] - lc[0] <= 0.006]
    lc.reset_index(drop=True,inplace=True)
    #lc = lc.iloc[:,[1,2,3,4]] / float(fluence_data.loc[trigger,'fluence'])
    lc.iloc[:,0] = pd.to_timedelta(pd.Series(lc.iloc[:,0]), unit = 's')
    lc.iloc[:,1] = pd.to_timedelta(pd.Series(lc.iloc[:,1]), unit = 's')
    #print(lc.iloc[:,1] - lc.iloc[:,0])
    lc = lc.resample('64ms', on=0).mean().pad()
    lc.reset_index(drop=True,inplace=True)
    lc = lc.loc[:,[2,3,4,5]] / float(fluence_data.loc[trigger,'fluence'])
    return len(lc), lc # Return length and the cut lightcurve
trig = 107
print(duration_data.loc[trig,'T90']/0.064)
print(cut_norm_lc(f'BATSE/GRB{trig}.fits'))
plt.plot(cut_norm_lc(f'BATSE/GRB{trig}.fits')[1])

In [ ]:
fluence_data = pd.read_pickle("DataFrames/fluence_data_BATSE.dat")
duration_data = pd.read_pickle("DataFrames/duration_data_BATSE.dat")
from scipy.signal import lfilter
def cut_norm_lc(filename): #Prepare single light curve, cut to T90 and normalize by fluence
    trigger = int(filename[6:-4])
    #Cut lightcurve
    file = pd.read_csv(filename, skiprows=1, header=None, sep='\s+')
    times = pd.DataFrame(fits.open(f'BATSE/GRB{trigger}.fits')[2].data.field('TIMES'))
    header = pd.Series(data=file.loc[0].values, index=['trig#', 'npts', 'nlasc', '1preb'], dtype='int64')
    trig_time = header['nlasc'] + 32
    lc = file.drop(0).reset_index(drop=True)
    for i in range(4):
        b1 = lc.iloc[0,i]
        b2 = lc.iloc[-1,i]
        lc.iloc[:,i] = lc.iloc[:,i] - [b1 + (b2-b1)/(len(lc)-1)*i for i in range(len(lc))]
    start = trig_time + max([times.min().min(), duration_data.loc[trigger,'start_T90']])/0.064
    end = trig_time + min([times.max().max(), duration_data.loc[trigger,'start_T90']+duration_data.loc[trigger,'T90']])/0.064 - 1
    #lc = lc.loc[start:end]
    lc = lc.loc[(trig_time + duration_data.loc[trigger,'start_T90']/0.064):(trig_time + (duration_data.loc[trigger,'start_T90'] + duration_data.loc[trigger,'T90'])/0.064 - 1)]
    lc.reset_index(drop=True,inplace=True)
    lc = lc.iloc[:,[0,1,2,3]] / float(fluence_data.loc[trigger,'fluence'])
    #for i in range(4):
        #lc.iloc[:,i] -= lc.iloc[:,i].min()
    return len(lc), lc # Return length and the cut lightcurve

filename = 'LC/GRB5508.csv'
display(duration_data.loc[int(filename[6:-4])])
display(fluence_data.loc[int(filename[6:-4]), 'fluence'])
length, lc = cut_norm_lc(filename)
#print(lc)
plt.plot(lc)

In [ ]:
plt.plot(abs(np.fft.rfft(pd.read_pickle('non_fft_dataset.dat'))))
plt.show()
plt.plot(abs(np.fft.rfft(pd.read_pickle('non_fft_dataset_Fermi.dat'))))
plt.show()
plt.plot(abs(np.fft.rfft(pd.read_pickle('non_fft_dataset_BATSE_LC.dat'))))
plt.show()

In [ ]:
from sklearn.preprocessing import Normalizer, StandardScaler, MinMaxScaler, MaxAbsScaler, RobustScaler
plt.plot(abs(np.fft.rfft(StandardScaler().fit_transform(pd.read_pickle('non_fft_dataset_BATSE_LC.dat')))))
plt.show()

In [ ]:
df1 = abs(np.fft.rfft(pd.read_pickle('non_fft_dataset.dat')))
print(df1)
df2 = abs(np.fft.rfft(pd.read_pickle('non_fft_dataset_Fermi.dat')))
print(df2)
df3 = abs(np.fft.rfft(pd.read_pickle('non_fft_dataset_BATSE_LC.dat')))
print(df3)

In [ ]:
x_max = 0
x_i = 0
for i in range(len(df1)):
    plt.plot(df1[i])
plt.show()
for i in range(len(df2)):
    if max(df2[i]) > x_max:
        x_max = max(df2[i])
        x_i = i
    plt.plot(df2[i])
plt.show()
print(x_i)
plt.plot(df2[x_i])
plt.show()
for i in range(len(df3)):
    plt.plot(df3[i])
plt.show()

In [ ]:
pd.read_pickle('non_fft_dataset_Fermi.dat').iloc[12]

In [ ]:
duration_data = pd.read_pickle('DataFrames/duration_data.dat')
fluence_data = pd.read_pickle('DataFrames/fluence_data.dat')

def cut_norm_lc(filename): #Prepare single light curve, cut to T100 and normalize by fluence
    grbname = filename[12:-7]
    #Cut lightcurve
    lc = pd.read_csv(filename, sep = ' ', header = None)
    lc = lc.loc[:, [0, 1, 3, 5, 7]]
    lc = lc.loc[lc.loc[:,0].apply(lambda x: duration_data.T100_start[grbname] <= x and x <= duration_data.T100_end[grbname])]
    lc.reset_index(drop=True,inplace=True)
    lc = lc.iloc[:,[1,2,3,4]] / float(fluence_data.loc[grbname, 'fluence'])
    return len(lc), lc # Return length and the cut lightcurve

for i in range(100):
    name = pd.read_pickle('non_fft_dataset.dat').index[i]
    print(name)
    plt.plot(df1[i])
    plt.show()
    plt.plot(cut_norm_lc(f'LightCurves/{name}_lc.dat')[1])
    plt.show()

In [ ]:
fluence_data = pd.read_csv('DataFrames/fluence_data_Fermi.csv', index_col=0)
def normalize_lc(filename): # Normalize by fluence
    grbname = filename[8:-4]
    #Cut lightcurve
    lc = pd.DataFrame(fits.open(filename)[2].data.field('NRGFLUX'))
    lc = lc.squeeze() / float(fluence_data.loc[f'GRB{grbname}','fluence'])
    return len(lc), lc # Return length and the normalized lightcurve

for i in range(100):
    name = pd.read_pickle('non_fft_dataset_Fermi.dat').index[i][3:]
    print('GRB'+name)
    plt.plot(df2[i])
    plt.show()
    plt.plot(normalize_lc(f'Fermi/bn{name}.fit')[1])
    plt.show()

In [ ]:
duration_data = pd.read_pickle("DataFrames/duration_data_BATSE.dat")
fluence_data = pd.read_pickle("DataFrames/fluence_data_BATSE.dat")

def cut_norm_lc(filename): #Prepare single light curve, cut to T90 and normalize by fluence
    trigger = int(filename[6:-4])
    #Cut lightcurve
    file = pd.read_csv(filename, skiprows=1, header=None, sep='\s+')
    times = pd.DataFrame(fits.open(f'BATSE/GRB{trigger}.fits')[2].data.field('TIMES'))
    header = pd.Series(data=file.loc[0].values, index=['trig#', 'npts', 'nlasc', '1preb'], dtype='int64')
    trig_time = header['nlasc'] + 32
    lc = file.drop(0).reset_index(drop=True)
    for i in range(4):
        b1 = lc.iloc[0,i]
        b2 = lc.iloc[-1,i]
        lc.iloc[:,i] = lc.iloc[:,i] - [b1 + (b2-b1)/(len(lc)-1)*i for i in range(len(lc))]
    start = trig_time + max([times.min().min(), duration_data.loc[trigger,'start_T90']])/0.064
    end = trig_time + min([times.max().max(), duration_data.loc[trigger,'start_T90']+duration_data.loc[trigger,'T90']])/0.064 - 1
    lc = lc.loc[start:end]
    #lc = lc.loc[(trig_time + duration_data.loc[trigger,'start_T90']/0.064):(trig_time + (duration_data.loc[trigger,'start_T90'] + duration_data.loc[trigger,'T90'])/0.064 - 1)]
    lc.reset_index(drop=True,inplace=True)
    lc = lc.iloc[:,[0,1,2,3]] / float(fluence_data.loc[trigger,'fluence'])
    return len(lc), lc # Return length and the cut lightcurve


for i in range(1500,1600):
    name = pd.read_pickle('non_fft_dataset_BATSE_LC.dat').index[i]
    print('GRB'+name)
    plt.plot(df3[i])
    plt.show()
    plt.plot(cut_norm_lc(f'LC/GRB{name}.csv')[1])
    plt.show()

In [ ]:
duration_data = pd.read_pickle("DataFrames/duration_data_BATSE.dat")
fluence_data = pd.read_pickle("DataFrames/fluence_data_BATSE.dat")
from math import ceil, floor
def cut_norm_lc(filename): #Prepare single light curve, cut to T90 and normalize by fluence
    trigger = int(filename[6:-4])
    #Cut lightcurve
    file = pd.read_csv(filename, skiprows=1, header=None, sep='\s+')
    times = pd.DataFrame(fits.open(f'BATSE/GRB{trigger}.fits')[2].data.field('TIMES'))
    header = pd.Series(data=file.loc[0].values, index=['trig#', 'npts', 'nlasc', '1preb'], dtype='int64')
    trig_time = header['nlasc'] + 32
    print(trig_time)
    lc = file.drop(0).reset_index(drop=True)
    for i in range(4):
        b1 = lc.iloc[0,i]
        b2 = lc.iloc[-1,i]
        lc.iloc[:,i] = lc.iloc[:,i] - [b1 + (b2-b1)/(len(lc)-1)*i for i in range(len(lc))]
        #lc.iloc[:,i] -= b2
    print(ceil(min([times.max().max(), duration_data.loc[trigger,'start_T90']+duration_data.loc[trigger,'T90']])/0.064))
    start = trig_time + floor(max([times.min().min(), duration_data.loc[trigger,'start_T90']])/0.064)
    end = trig_time + ceil(min([times.max().max(), duration_data.loc[trigger,'start_T90']+duration_data.loc[trigger,'T90']])/0.064) - 1
    lc = lc.loc[start:end]
    print(lc)
    #lc = lc.loc[(trig_time + duration_data.loc[trigger,'start_T90']/0.064 - 1):(trig_time + (duration_data.loc[trigger,'start_T90'] + duration_data.loc[trigger,'T90'])/0.064)]
    lc.reset_index(drop=True,inplace=True)
    #lc = lc.iloc[:,[0,1,2,3]] / float(fluence_data.loc[trigger,'fluence'])
    return len(lc), lc # Return length and the cut lightcurve

i = -1
for x in pd.read_pickle('non_fft_dataset_BATSE_LC.dat').index:
    i += 1
    if x == '491':
        break
        
name = pd.read_pickle('non_fft_dataset_BATSE_LC.dat').index[i]
print('GRB'+name)
print(duration_data.loc[int(name)])
plt.plot(df3[i])
plt.show()
plt.plot(cut_norm_lc(f'LC/GRB{name}.csv')[1])
plt.show()

In [ ]:
filename = 'LC/GRB1009.csv'
trigger = int(filename[6:-4])
display(duration_data.loc[trigger])
file = pd.read_csv(filename, skiprows=1, header=None, sep='\s+')
header = pd.Series(data=file.loc[0].values, index=['trig#', 'npts', 'nlasc', '1preb'], dtype='int64')
trig_time = header['nlasc'] + 32
lc = file.drop(0).reset_index(drop=True)
lc = lc.iloc[:,[0,1,2,3]] / float(fluence_data.loc[trigger,'fluence'])
plt.plot(lc)

In [ ]:
df1 = pd.read_csv('DataFrames/classification_BATSE.txt',index_col=0)
df1.index = df1.index.astype(str)
print(df1.index)
df2 = pd.read_pickle('non_fft_dataset_BATSE_LC.dat')
print(df2.index)
df = pd.concat((df1,df2),axis=1)
#df[6645]
print(df[df.isna().any(1)])
with pd.option_context('display.max_rows', None):
    display(df)

In [ ]:
trigger = 148
duration_data = pd.read_pickle("DataFrames/duration_data_BATSE.dat")
display(duration_data.loc[trigger])
trigger_time = header['nlasc'] + 32
#rates = rates.loc[rates.reset_index().loc.apply(lambda x: trigger_time + duration_data.loc[trigger,'start_T90']/0.064 <= x and x <= trigger_time + (duration_data.loc[trigger,'start_T90'] + duration_data.loc[trigger,'T90'])/0.064)]
duration_data.loc[trigger]
rates.loc[(trigger_time + duration_data.loc[trigger,'start_T90']/0.064 - 3):(trigger_time + (duration_data.loc[trigger,'start_T90'] + duration_data.loc[trigger,'T90'])/0.064 - 1)]

In [ ]:
duration_data = pd.read_pickle("DataFrames/duration_data_BATSE.dat")
fluence_data = pd.read_pickle("DataFrames/fluence_data_BATSE.dat")

def cut_norm_lc(filename): #Prepare single light curve, cut to T90 and normalize by fluence
    trigger = int(filename[6:-4])
    #Cut lightcurve
    file = pd.read_csv(filename, skiprows=1, header=None, sep='\s+')
    header = pd.Series(data=file.loc[0].values, index=['trig#', 'npts', 'nlasc', '1preb'], dtype='int64')
    trig_time = header['nlasc'] + 32
    lc = file.drop(0).reset_index(drop=True)
    lc = lc.loc[(trig_time + duration_data.loc[trigger,'start_T90']/0.064):(trig_time + (duration_data.loc[trigger,'start_T90'] + duration_data.loc[trigger,'T90'])/0.064 - 1)]
    lc.reset_index(drop=True,inplace=True)
    lc = lc.iloc[:,[0,1,2,3]] / float(fluence_data.loc[trigger,'fluence'])
    return len(lc), lc # Return length and the cut lightcurve

cut_norm_lc('LC/GRB105.csv')

In [ ]:
plt.hist(np.log10(duration_data.loc[5585:,'T90']), bins = 50, color = 'C0', edgecolor = 'C0', alpha = 0.5)

In [ ]:
short = pd.read_csv('DataFrames/classification_BATSE_Short.txt', index_col=0)
#short.index = short.index.str.slice(3).astype(int)
print(short.index)
#print(pd.read_csv("embedding_TSNE_BATSE_Short.csv", header = None, index_col = 0).index)
#short.index.isin(pd.read_csv("embedding_TSNE_BATSE_Short.csv", header = None, index_col = 0).index.tolist())

In [ ]:
#short.loc[pd.read_csv("embedding_TSNE_BATSE_Short.csv", header = None, index_col = 0).index]
#print(pd.read_csv("embedding_TSNE_BATSE_Short.csv", header = None, index_col = 0).index)
pd.concat([short, pd.read_csv("embedding_TSNE_BATSE_Short.csv", header = None, index_col = 0)], axis=1).dropna(subset=[1])

In [ ]:
import sklearn.cluster as cluster
prediction = cluster.SpectralClustering(2).fit_predict(abs(np.fft.rfft(pd.read_pickle('non_fft_dataset.dat'))))
print(prediction)
t1 = 0
t2 = 0
for i in prediction:
    print(i)
    if(i == 0):
        t1 += 1
    else:
        t2 += 1
print(t1)
print(t2)

In [ ]:
emb = pd.read_csv("embedding_TSNE_20.csv", header = None).set_index(0)
plt.scatter(emb[1], emb[2], c=prediction)

In [ ]:
#path = 'Dropbox/jackson/maps_0524/tsne_maps/'
#path = 'Dropbox/jackson/tsne_maps_swift/'
namedict = pd.read_pickle('DataFrames/classification.dat').reset_index().to_dict()['GRBname']
plt.rcParams["axes.prop_cycle"] = plt.cycler("color", plt.cm.Dark2.colors)
for file in os.listdir(path):
    emb = pd.read_csv((path+file), header = None).replace({0:namedict}).set_index(0)
    plt.scatter(emb[1], emb[2], s=10, c=pd.read_pickle('DataFrames/classification.dat').replace({'L': 'C2', 'S': 'C1'}).type.reindex_like(emb))
    plt.tick_params('both',bottom=False,top=False,left=False,right=False,labelbottom=False,labelleft=False)
    plt.show()

In [ ]:
path = 'umap_maps/'
classification = pd.read_pickle('DataFrames/classification.dat')
classification['GRB090813'] = 'S'
namedict = classification.reset_index().to_dict()['GRBname']
plt.rcParams["axes.prop_cycle"] = plt.cycler("color", plt.cm.Dark2.colors)
for file in os.listdir(path):
    emb = pd.read_csv((path+file), header = None).replace({0:namedict}).set_index(0)
    plt.scatter(emb[1], emb[2], s=10, c=classification.replace({'L': 'C2', 'S': 'C1'}).type.reindex_like(emb))
    plt.tick_params('both',bottom=False,top=False,left=False,right=False,labelbottom=False,labelleft=False)
    plt.show()

In [ ]:
path = 'tsne_maps/'
for file in os.listdir(path):
    emb = pd.read_csv((path+file), header = None).set_index(0)
    plt.scatter(emb[1], emb[2], c=emb[5])
    plt.show()

In [ ]:
path = 'umap_maps/'
for file in os.listdir(path):
    emb = pd.read_csv((path+file), header = None).set_index(0)
    plt.scatter(emb[1], emb[2], c=emb[4])
    plt.show()

In [ ]:
pd.read_pickle('DataFrames/classification.dat').reset_index()

In [ ]:
hardness_data = pd.read_pickle('DataFrames/hardness_data.dat')
classification = pd.read_csv('DataFrames/classification_Swift.txt')
group = classification.groupby(classification.type)
short = group.get_group('S')
long = group.get_group('L')
plt.hist(np.log10(hardness_data.loc[short.GRBname,'hardness']), range = (-1,1), bins = 25, color = 'C1', edgecolor = 'C1', alpha = 0.5)
plt.hist(np.log10(hardness_data.loc[long.GRBname,'hardness']), range = (-1,1), bins = 25, color = 'C2', edgecolor = 'C2', alpha = 0.5)
plt.xlabel('$s(50-100keV)/s(25-50keV)$', size = 12)

In [ ]:
hardness_data = pd.read_pickle('DataFrames/hardness_data_BATSE.dat')
classification = pd.read_csv('DataFrames/classification_BATSE.txt')
group = classification.groupby(classification.type)
short = group.get_group('S')
long = group.get_group('L')
plt.hist(np.log10(hardness_data.loc[short.GRBname,'hardness']), range = (-1,1), bins = 25, color = 'C1', edgecolor = 'C1', alpha = 0.5)
plt.hist(np.log10(hardness_data.loc[long.GRBname,'hardness']), range = (-1,1), bins = 25, color = 'C2', edgecolor = 'C2', alpha = 0.5)
plt.xlabel('$s(100-300keV)/s(50-100keV)$', size = 12)
#plt.xlabel('$s(50-100keV)/s(20-50keV)$', size = 12)

In [ ]:
intersection = pd.read_csv('DataFrames/intersection_Swift_Fermi.csv')
Swift = pd.read_csv('DataFrames/classification_Swift.txt')
Swift = Swift[Swift.isin(intersection.Swift.tolist()).GRBname].reset_index(drop=True)
display(Swift)
Fermi = pd.read_csv('DataFrames/classification_Fermi.txt')
Fermi = Fermi[Fermi.isin(intersection.Fermi.tolist()).GRBname].reset_index(drop=True)
display(Fermi)
classification = pd.concat([Swift,Fermi], axis=1)
classification = classification[classification.iloc[:,1] != classification.iloc[:,3]].reset_index(drop=True)
classification.columns = ['Swift', 'type', 'Fermi', 'type']
display(classification)
classification.set_index('Swift').to_csv('DataFrames/mismatching_classifications.csv')

In [ ]:
pd.__version__

In [ ]:
emb = pd.read_csv("embedding_TSNE_20.csv", header = None).set_index(0)
source = ColumnDataSource({'names': emb.index.tolist(), 'x': emb[1].tolist(), 'y': emb[2].tolist()})#, 'color': None})
plot = figure(title='t-SNE projection of the BATSE catalog of GRBs', plot_width=800, plot_height=600, tools="pan,wheel_zoom,box_zoom,lasso_select,reset")
plot.add_tools(HoverTool(tooltips="""
<div>
    <div>
        <span><pre>@names</pre></span>
    </div>
</div>
"""))
#color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8)
#plot.add_layout(color_bar, 'above')
plot.scatter(x = 'x', y = 'y', source = source)#, color={'field': 'color', 'transform': color_mapper})
multi_choice = MultiChoice(options=emb.index.tolist())
update_highlights = CustomJS(args=dict(source=source), code="""
    source.selected['indices'] = []
    for(var i = 0; i < source.get_length(); i++) {
        //console.log(i, source.data['names'][i], cb_obj.value)
        if (cb_obj.value.includes(source.data['names'][i])) {
            source.selected['indices'].push(i)
        }
    }
    source.change.emit()
""")
selected_indices = []
multi_choice.js_on_change('value', update_highlights)
callback = CustomJS(args=dict(source=source), code="""
    //console.log('Running CustomJS callback now.');
    var indices = source.selected.indices;
    var kernel = IPython.notebook.kernel;
    kernel.execute("selected_indices = " + indices)
""")
plot.js_on_event('selectiongeometry', callback)

In [ ]:
show(column(multi_choice,plot))

In [ ]:
selected = emb.reset_index().isin(emb.iloc[list(selected_indices)].index)[0].replace({False: 'L', True: 'S'})
df = pd.concat([pd.Series(emb.index), selected], axis = 1, ignore_index = True).rename({0: 'GRBname', 1: 'type'}, axis=1).set_index('GRBname')
print(df.loc[df.type=='S'])
print(df.loc[df.type=='L'])
print(len(df))
df = df.loc[df.type=='S'].replace({'S':'C0'})
df
#df.to_csv('DataFrames/third_group_Swift.txt')

In [ ]:
emb = pd.read_csv("embedding_TSNE.csv", header = None).set_index(0)
emb[3] = DBSCAN(eps=7).fit_predict(emb.loc[:,[1,2]])
#emb[3] = emb[3].map({0:'C2',1:'C1'})
plt.scatter(emb[1], emb[2], s=5, c=emb[3])
plt.show()

In [ ]:
hardness_data = pd.read_pickle('DataFrames/hardness_data_BATSE.dat')
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(hardness_data)

In [ ]:
classification = pd.concat([pd.read_csv('DataFrames/classification_BATSE.txt', index_col=0), pd.read_csv("embedding_TSNE_BATSE.csv", header = None, index_col = 0)], axis=1).dropna(subset=[1])
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(classification)

In [ ]:
non_fft_dataset = pd.read_pickle('non_fft_dataset_BATSE_LC.dat')
non_fft_dataset

In [ ]:
emb = pd.read_csv('embedding_TSNE_BATSE.csv', header=None, index_col=0)
emb

In [ ]:
np.vstack((non_fft_dataset.index, emb.loc[:,1], emb.loc[:,2])).T

In [ ]:
path = 'Dropbox/jackson/tsne_maps_fermi/'
counts = {grb:[0,0] for grb in pd.read_pickle('non_fft_dataset_Fermi.dat').index}
#for file in os.listdir(path):
for i in range(100):
    print(i)
    file = f'embedding_TSNE_Fermi{i}.csv'
    emb = pd.read_csv(path+file,header=None,index_col=0)
    long = int(len(emb.groupby(emb.iloc[:,2]).get_group(1)) > len(emb.groupby(emb.iloc[:,2]).get_group(0)))
    emb.iloc[:,2] = emb.replace({long:1,1-long:0}).iloc[:,2]
    for i in range(emb.index.size):
        row = emb.iloc[i]
        counts[row.name][int(row[3])] += 1
print(counts)

In [ ]:
df = pd.DataFrame(counts).T
df[3] = df.max(axis=1)/df.sum(axis=1)
df.columns = ['S', 'L', '%']
display(df)

In [ ]:
emb = pd.read_csv('Dropbox/jackson/tsne_maps_fermi/embedding_TSNE_Fermi3.csv', header=None, index_col=0)
long = int(len(emb.groupby(emb.iloc[:,2]).get_group(1)) > len(emb.groupby(emb.iloc[:,2]).get_group(0)))
emb.iloc[:,2] = emb.replace({long:1,1-long:0}).iloc[:,2]
#display(emb)
for i in range(emb.index.size):
    row = emb.iloc[i]
    counts[row.name][int(row[3])] += 1
print(counts)

In [ ]:
emb = pd.read_csv('Dropbox/jackson/tsne_maps_swift/embedding_TSNE_Swift6.csv', header=None, index_col=0)
plt.scatter(emb[1],emb[2],s=5,c=emb[3])

In [ ]:
df = pd.read_pickle('non_fft_dataset.dat')
with pd.option_context('display.max_rows', None):
    display(df)

In [ ]:
print(pd.read_csv('DataFrames/classification_Swift.txt'))

In [ ]:
classification = pd.read_csv('DataFrames/classification_Swift.txt')
df = pd.read_pickle('non_fft_dataset.dat').loc[classification.GRBname.loc[classification.type=='S']]
df = df.loc[:, (df != 0).any(axis=0)]
fft = np.fft.rfft(df)

In [ ]:
duration_data = pd.read_pickle("DataFrames/duration_data.dat")
emb = TSNE(verbose = 2, perplexity = 15, n_iter = 15000).fit_transform(abs(fft))
plt.scatter(emb[:,0], emb[:,1], s=5, c=np.log10(duration_data.loc[df.index].T90), cmap='plasma')
plt.show()

In [ ]:
color = np.log10(duration_data.loc[df.index].T90)
source = ColumnDataSource({'names': df.index.tolist(), 'x': emb[:,0].tolist(), 'y': emb[:,1].tolist(), 'color': color})
color_mapper = LinearColorMapper(palette=Plasma256, low=min(color), high=max(color))
plot = figure(title='t-SNE projection of the Swift catalog of GRBs', plot_width=800, plot_height=600, tools="pan,wheel_zoom,box_zoom,lasso_select,reset")
plot.add_tools(HoverTool(tooltips="""
<div>
    <div>
        <span><pre>@names</pre></span>
    </div>
</div>
"""))
plot.scatter(x = 'x', y = 'y', source = source, color={'field': 'color', 'transform': color_mapper})
show(plot)

In [ ]:
classification = pd.read_csv('DataFrames/classification_Fermi.txt')
df = pd.read_pickle('non_fft_dataset_Fermi.dat').loc[classification.GRBname.loc[classification.type=='S']]
df = df.loc[:, (df != 0).any(axis=0)]
fft = np.fft.rfft(df)

In [ ]:
duration_data = pd.read_pickle("DataFrames/duration_data_Fermi.dat")
emb = TSNE(verbose = 2, perplexity = 20, n_iter = 15000).fit_transform(abs(fft))
plt.scatter(emb[:,0], emb[:,1], s=5, c=np.log10(duration_data.loc[df.index].T90), cmap='plasma')
plt.show()

In [ ]:
color = np.log10(duration_data.loc[df.index].T90)
source = ColumnDataSource({'names': df.index.tolist(), 'x': emb[:,0].tolist(), 'y': emb[:,1].tolist(), 'color': color})
color_mapper = LinearColorMapper(palette=Plasma256, low=min(color), high=max(color))
plot = figure(title='t-SNE projection of the Fermi catalog of GRBs', plot_width=800, plot_height=600, tools="pan,wheel_zoom,box_zoom,lasso_select,reset")
plot.add_tools(HoverTool(tooltips="""
<div>
    <div>
        <span><pre>@names</pre></span>
    </div>
</div>
"""))
plot.scatter(x = 'x', y = 'y', source = source, color={'field': 'color', 'transform': color_mapper})
show(plot)

In [ ]:
from scipy.spatial.distance import cdist
emb = pd.read_csv('embedding_TSNE_Fermi_20.csv', header=None, index_col=0)
classification = pd.read_csv('DataFrames/classification_Fermi.txt')
group = classification.groupby(classification.type)
short = emb.loc[group.get_group('S').GRBname]
long = emb.loc[group.get_group('L').GRBname]
print(cdist(short,long).min())

In [ ]:
#df = pd.read_pickle('non_fft_dataset_BATSE_LC_2.dat')
#df[~df.isin(df.dropna())]
#df[df.eq(np.inf).any(1)]
df = pd.read_pickle('DataFrames/fluence_data.dat')
#df[df.astype(float)==0]
#df[df.eq(0).any(1)]

In [ ]:
emblist = []
non_fft_dataset = pd.read_pickle('non_fft_dataset.dat')
fft = np.fft.rfft(non_fft_dataset)
warnings.filterwarnings('ignore')
from ipynb.fs.full.config_TSNE import tsne_params
verbose, perplexity, n_iter = tsne_params['verbose'], tsne_params['perplexity'], tsne_params['n_iter']
for i in range(16):
    print(f'embedding #{i+1} out of 16:')
    emb = TSNE(verbose = verbose, perplexity = perplexity, n_iter = n_iter).fit_transform(abs(fft))
    emblist.append(pd.DataFrame(np.vstack((emb[:,0], emb[:,1])).T, non_fft_dataset.index))

In [ ]:
from ipynb.fs.full.config_TSNE import conf

classification = pd.read_pickle('DataFrames/classification.dat')
plt.rcParams["axes.prop_cycle"] = plt.cycler("color", plt.cm.Dark2.colors)
color = classification.replace({'L': 'C2', 'S': 'C1'}).type

fig = plt.figure(figsize=(8,8), tight_layout=True)
#fig.suptitle('Swift maps with randomized orders', size=16)
gs = fig.add_gridspec(4, 4, hspace=0, wspace=0)
axs = gs.subplots(sharex='col', sharey='row')
for i in range(16):
    ax = axs[i//4][i%4]
    emb = emblist[i]
    ax.set(xticks=(), yticks=())
    #ax.scatter(emb[0], emb[1], s=conf['radius'], c=conf['color'], cmap=conf['cmap'])
    ax.scatter(emb[0], emb[1], s=2, c=color, cmap=conf['cmap'])
fig.savefig("changeorder.jpg", dpi=300)
fig.show()

In [ ]:
duration_data_s = pd.read_pickle('DataFrames/duration_data.dat')
classification = pd.read_csv('DataFrames/classification_Swift.txt')
group = classification.groupby(classification.type)
swift_short = group.get_group('S')
swift_long = group.get_group('L')
swift_s = [d for d in duration_data_s.loc[swift_short.GRBname,'T90']]
swift_l = [d for d in duration_data_s.loc[swift_long.GRBname,'T90']]

duration_data_f = pd.read_pickle('DataFrames/duration_data_Fermi.dat')
classification = pd.read_csv('DataFrames/classification_Fermi.txt')
group = classification.groupby(classification.type)
fermi_short = group.get_group('S')
fermi_long = group.get_group('L')
fermi_s = [d for d in duration_data_f.loc[fermi_short.GRBname,'T90']]
fermi_l = [d for d in duration_data_f.loc[fermi_long.GRBname,'T90']]

duration_data_b = pd.read_pickle('DataFrames/duration_data_BATSE.dat')
classification = pd.read_csv('DataFrames/classification_BATSE.txt')
group = classification.groupby(classification.type)
batse_short = group.get_group('S')
batse_long = group.get_group('L')
batse_s = [d for d in duration_data_b.loc[batse_short.GRBname,'T90']]
batse_l = [d for d in duration_data_b.loc[batse_long.GRBname,'T90']]

fig, axs = plt.subplots(3,figsize=(8,6),constrained_layout=True)
#figure out how to do logarithmic ticks
plt.xlabel('log$T_{90}$',size=14)
plt.ylabel('Count',size=14)
axs[0].set_title('Swift',size=16)
axs[0].hist(np.log10(duration_data_s.loc[swift_short.GRBname,'T90']), range = (-3,4), bins = 45, color = 'C1', edgecolor = 'C1', alpha = 0.5)
axs[0].hist(np.log10(duration_data_s.loc[swift_long.GRBname,'T90']), range = (-3,4), bins = 45, color = 'C2', edgecolor = 'C2', alpha = 0.5)
axs[0].legend(['short','long'], loc='upper left')
axs[1].set_title('Fermi',size=16)
axs[1].hist(np.log10(duration_data_f.loc[fermi_short.GRBname,'T90']), range = (-3,4), bins = 45, color = 'C1', edgecolor = 'C1', alpha = 0.5)
axs[1].hist(np.log10(duration_data_f.loc[fermi_long.GRBname,'T90']), range = (-3,4), bins = 45, color = 'C2', edgecolor = 'C2', alpha = 0.5)
axs[1].legend(['short','long'], loc='upper left')
axs[2].set_title('BATSE',size=16)
axs[2].hist(np.log10(duration_data_b.loc[batse_short.GRBname,'T90']), range = (-3,4), bins = 45, color = 'C1', edgecolor = 'C1', alpha = 0.5)
axs[2].hist(np.log10(duration_data_b.loc[batse_long.GRBname,'T90']), range = (-3,4), bins = 45, color = 'C2', edgecolor = 'C2', alpha = 0.5)
axs[2].legend(['short','long'], loc='upper left')
#plt.savefig('duration_comparison.jpg',dpi=300)

In [ ]:
hardness_data_s = pd.read_pickle('DataFrames/hardness_data.dat')
hardness_data_b = pd.read_pickle('DataFrames/hardness_data_BATSE.dat')

fig, axs = plt.subplots(2,1,figsize=(4,8),constrained_layout=True)
axs[0].set_title('Swift',size=16)
axs[0].set_xlabel('log$T_{90}$',size=12)
axs[0].set_ylabel('$s(50-100)/s(25-50)$',size=12)
#axs[0].scatter(np.log10(duration_data_s.loc[swift_short.GRBname,'T90']), hardness_data_s.loc[swift_short.GRBname,'hardness'], c='C1', marker='x')
#axs[0].scatter(np.log10(duration_data_s.loc[swift_long.GRBname,'T90']), hardness_data_s.loc[swift_long.GRBname,'hardness'], c='C2', marker='x')
s1 = sns.kdeplot(x=np.log10(duration_data_s.loc[swift_short.GRBname,'T90']), y=hardness_data_s.loc[swift_short.GRBname,'hardness'], bw_adjust=0.6, clip=((-2,3),(-0.1,5.2)), levels=5, thresh=0.1, ax=axs[0], color=plt.cm.Dark2.colors[1], fill=False, alpha=0.75)
s2 = sns.kdeplot(x=np.log10(duration_data_s.loc[swift_long.GRBname,'T90']), y=hardness_data_s.loc[swift_long.GRBname,'hardness'], bw_adjust=0.6, clip=((-2,3),(-0.1,5.2)), levels=5, thresh=0.1, ax=axs[0], color=plt.cm.Dark2.colors[2], fill=False, alpha=0.75)
#axs[0].legend(['short','long'], loc='upper right')
axs[1].set_title('BATSE',size=16)
axs[1].set_xlabel('log$T_{90}$',size=12)
axs[1].set_ylabel('$s(50-100)/s(20-50)$',size=12)
#axs[1].scatter(np.log10(duration_data_b.loc[batse_short.GRBname,'T90']), hardness_data_b.loc[batse_short.GRBname,'hardness'], c='C1', marker='x')
#axs[1].scatter(np.log10(duration_data_b.loc[batse_long.GRBname,'T90']), hardness_data_b.loc[batse_long.GRBname,'hardness'], c='C2', marker='x')
b1 = sns.kdeplot(x=np.log10(duration_data_b.loc[batse_short.GRBname,'T90']), y=hardness_data_b.loc[batse_short.GRBname,'hardness'], bw_adjust=0.5, clip=((-2,3),(-0.1,5.2)), levels=5, thresh=0.1, ax=axs[1], color=plt.cm.Dark2.colors[1], fill=False, alpha=0.75)
b2 = sns.kdeplot(x=np.log10(duration_data_b.loc[batse_long.GRBname,'T90']), y=hardness_data_b.loc[batse_long.GRBname,'hardness'], bw_adjust=0.5, clip=((-2,3),(-0.1,5.2)), levels=5, thresh=0.1, ax=axs[1], color=plt.cm.Dark2.colors[2], fill=False, alpha=0.75)
#axs[1].legend(['short','long'], loc='upper right')
plt.savefig('DataFrames/hardness_comparison.jpg',dpi=300)

In [ ]:
duration_data_s = pd.read_pickle('DataFrames/duration_data.dat')
classification = pd.read_csv('DataFrames/classification_Swift.txt')
group = classification.groupby(classification.type)
swift_short = group.get_group('S')
swift_long = group.get_group('L')
swift_s = [d for d in duration_data_s.loc[swift_short.GRBname,'T90']]
swift_l = [d for d in duration_data_s.loc[swift_long.GRBname,'T90']]

duration_data_f = pd.read_pickle('DataFrames/duration_data_Fermi.dat')
classification = pd.read_csv('DataFrames/classification_Fermi.txt')
group = classification.groupby(classification.type)
fermi_short = group.get_group('S')
fermi_long = group.get_group('L')
fermi_s = [d for d in duration_data_f.loc[fermi_short.GRBname,'T90']]
fermi_l = [d for d in duration_data_f.loc[fermi_long.GRBname,'T90']]

duration_data_b = pd.read_pickle('DataFrames/duration_data_BATSE.dat')
classification = pd.read_csv('DataFrames/classification_BATSE.txt')
group = classification.groupby(classification.type)
batse_short = group.get_group('S')
batse_long = group.get_group('L')
batse_s = [d for d in duration_data_b.loc[batse_short.GRBname,'T90']]
batse_l = [d for d in duration_data_b.loc[batse_long.GRBname,'T90']]

fig, axs = plt.subplots(3,figsize=(8,6),constrained_layout=True)
#figure out how to do logarithmic ticks
plt.xlabel('log$T_{90}$', size=14)
axs[1].set_ylabel('Count (Normalized)', size=14)
axs[0].set_title('Swift', size=16)
axs[0].hist(np.log10(duration_data_s.loc[swift_short.GRBname,'T90']), density=True, range = (-3,4), bins = 45, color = 'C1', edgecolor = 'C1', alpha = 0.5)
axs[0].hist(np.log10(duration_data_s.loc[swift_long.GRBname,'T90']), density=True, range = (-3,4), bins = 45, color = 'C2', edgecolor = 'C2', alpha = 0.5)
axs[0].legend(['short','long'], loc='upper left')
axs[1].set_title('Fermi', size=16)
axs[1].hist(np.log10(duration_data_f.loc[fermi_short.GRBname,'T90']), density=True, range = (-3,4), bins = 45, color = 'C1', edgecolor = 'C1', alpha = 0.5)
axs[1].hist(np.log10(duration_data_f.loc[fermi_long.GRBname,'T90']), density=True, range = (-3,4), bins = 45, color = 'C2', edgecolor = 'C2', alpha = 0.5)
axs[1].legend(['short','long'], loc='upper left')
axs[2].set_title('BATSE', size=16)
axs[2].hist(np.log10(duration_data_b.loc[batse_short.GRBname,'T90']), density=True, range = (-3,4), bins = 45, color = 'C1', edgecolor = 'C1', alpha = 0.5)
axs[2].hist(np.log10(duration_data_b.loc[batse_long.GRBname,'T90']), density=True, range = (-3,4), bins = 45, color = 'C2', edgecolor = 'C2', alpha = 0.5)
axs[2].legend(['short','long'], loc='upper left')
for ax in axs:
    ax.set_xlim(-2, 3.5)
plt.savefig('duration_comparison_normalized.jpg',dpi=300)
plt.show()

In [ ]:
s = pd.read_pickle('DataFrames/stability_Swift.dat')
f = pd.read_pickle('DataFrames/stability_Fermi.dat')
b = pd.read_pickle('DataFrames/stability_BATSE.dat')

emb_s = pd.read_csv("embedding_TSNE_20.csv", header = None, index_col = 0)
emb_f = pd.read_csv("embedding_TSNE_Fermi_20.csv", header = None, index_col = 0)
emb_b = pd.read_csv("embedding_TSNE_BATSE.csv", header = None, index_col = 0)

fig, axs = plt.subplots(1,3,figsize=(12,4),constrained_layout=True)
fig.suptitle('Stability', size=18)
axs[0].set_title('Swift', size=14)
axs[0].scatter(emb_s[1], emb_s[2], s=5, c=s.loc[:,'%'])
axs[0].set(xticks=(), yticks=())
axs[1].set_title('Fermi', size=14)
axs[1].scatter(emb_f[1], emb_f[2], s=5, c=f.loc[:,'%'])
axs[1].set(xticks=(), yticks=())
axs[2].set_title('BATSE', size=14)
axs[2].scatter(emb_b[1], emb_b[2], s=5, c=b.loc[:,'%'])
axs[2].set(xticks=(), yticks=())
cbar = plt.colorbar(axs[2].get_children()[0], ax=axs[2], location = 'right')
cbar.set_label('$\max\{P(X \in L),P(X \in S)\}$', size=12)
#fig.savefig('stability_all.jpg', dpi=300)

In [ ]:
df = pd.read_table("summary/Basic_BATSE_new.dat", sep="\s+", comment='#', header=None, usecols=[0,2],index_col=0)
#df
#df[df[2].str.contains('-')]
for i in range(len(df)):
    df.iloc[i,0] = 'GRB' + df.iloc[i,0]
    if df.iloc[i,:].str.contains('-').iloc[0]:
        if df.iloc[i,0][:-1] == df.iloc[i-1,0][:-1]:
            df.iloc[i,0] = df.iloc[i,0][:-1] + ['A','B','C','D','E'][['A','B','C','D','E'].index(df.iloc[i-1,0][-1])+1]
        else:
            df.iloc[i,0] = df.iloc[i,0][:-1] + 'A'
df.columns = ['Name']
df.loc[2464] = 'GRB930725B'
df = df.loc[pd.read_csv('DataFrames/classification_BATSE.txt', index_col=0).index]
df.index.name = 'trig'
df = df.sort_values('trig')
for i in ['Other Name', 'Type', 'BATSE Type', 'BATSE Prob.', 'Swift Type', 'Swift Prob.', 'Fermi Type', 'Fermi Prob.']:
    df[i] = ''
df = df.set_index('Name')
display(df)
df.to_pickle('DataFrames/complete_catalog.dat')
#print(df.to_latex())
#print(df.set_index('name'))#[:,'name'])
#pd.read_csv('DataFrames/classification_BATSE.txt', index_col=0).loc[2464]
#df.to_pickle('DataFrames/names_from_trig_BATSE.dat')

In [ ]:
df = pd.read_pickle('DataFrames/complete_catalog.dat')
df = pd.concat((df,pd.read_csv('embedding_TSNE_20.csv',index_col=0,header=None).index.to_series()),0).drop(columns=[0])
df.index.name = 'Name'
df.columns = ['Other Name', 'Type', 'BATSE Type', 'BATSE Prob.', 'Swift Type', 'Swift Prob.', 'Fermi Type', 'Fermi Prob.']
for i in ['Other Name', 'Type', 'BATSE Type', 'BATSE Prob.', 'Swift Type', 'Swift Prob.', 'Fermi Type', 'Fermi Prob.']:
    df[i] = ''
display(df)
df.to_pickle('DataFrames/complete_catalog.dat')

In [ ]:
df = pd.read_pickle('DataFrames/complete_catalog.dat')
intersection = pd.read_csv('DataFrames/intersection_Swift_Fermi.csv')
for index, burst in enumerate(intersection.Swift):
    pass
    #df.loc[burst, 'Other Name'] = intersection.loc[index,'Fermi']
display(df)
df.to_pickle('DataFrames/complete_catalog.dat')

In [ ]:
table = pd.read_table('Downloads/Summary_table.txt', sep = '\s+', skiprows=[0,2], index_col='GRB_name', usecols = [0,1])
table.loc['GRB091024372'] = 'GRB091024A'
table.loc['GRB091024380'] = 'GRB091024B'
table.loc['GRB130925173'] = 'GRB130925'
#'GRB080916406'
table = table.loc[pd.read_csv('DataFrames/classification_Fermi.txt', index_col=0).index]
#table = table.loc[pd.read_csv('DataFrames/intersection_Swift_Fermi.csv', index_col='Fermi').index]
#

swift = pd.read_csv('DataFrames/intersection_Swift_Fermi.csv', index_col=0).index
for i in range(len(table)):
    if table.iloc[i,0][:-1] in swift and table.reset_index().iloc[i,0] == table.reset_index().iloc[swift.get_loc(table.iloc[i,0][:-1]),0]:
        print(table.iloc[i,0])
        table.iloc[i,0] = table.iloc[i,0][:-1]
for i in range(len(table)):
    if table.iloc[i,0][-1] == '*':
        table.iloc[i,0] = table.iloc[i,0][:-1]
for i in range(len(table)):
    if int(table.iloc[i,0][3:5]) < 10:
        if table.iloc[i,0][-1] == 'A' and table.iloc[i,0][:-1] != table.iloc[i+1,0][:-1]:
            table.iloc[i,0] = table.iloc[i,0][:-1]
table = table.reset_index()#.set_index('#')
table.columns = ['Other Name', 'Name']
table = table.set_index('Name')
display(table)
#table.loc[pd.read_csv('DataFrames/classification_Swift.txt', index_col=0).index]

In [ ]:
df = pd.read_pickle('DataFrames/complete_catalog.dat')
for index, burst in enumerate(table.index):
    df.loc[burst] = table.iloc[index,0]
for i in range(len(df)):
    name = df.index.tolist()[i]
    if int(name[3]) == 9:
        num = '19'
    else:
        num = '20'
    df = df.rename(index={name : (name[:3] + num + name[3:])})
df = df.sort_index(kind='stable')
for i in range(len(df)):
    name = df.index.tolist()[i]
    df = df.rename(index={name : name[:3] + name[5:]})
dupes = []
for i in range(len(df)-1):
    if df.index[i] == df.index[i+1][:-1]:
        print(df.index[i])
        dupes.append(df.index[i])
#df = df.drop(dupes)
df.index.name = 'Name'
df.columns = ['Other Name', 'Type', 'BATSE Type', 'BATSE Prob.', 'Swift Type', 'Swift Prob.', 'Fermi Type', 'Fermi Prob.']
for i in ['Type', 'BATSE Type', 'BATSE Prob.', 'Swift Type', 'Swift Prob.', 'Fermi Type', 'Fermi Prob.']:
    df[i] = ''
display(df)
df.to_pickle('DataFrames/complete_catalog.dat')

In [ ]:
df = pd.read_pickle('DataFrames/complete_catalog.dat')
names = pd.read_pickle('DataFrames/names_from_trig_BATSE.dat')
classification = pd.read_csv('DataFrames/classification_BATSE.txt', index_col=0)
stability = pd.read_pickle('DataFrames/stability_BATSE.dat')
mismatch = pd.read_csv('DataFrames/mismatching_bursts_BATSE.txt', index_col=0)
for trig in names.index:
    name = names.loc[trig,'Name']
    t = classification.loc[trig,'type']
    df.loc[name,'Type'] = t
    df.loc[name,'BATSE Type'] = t
    if stability.loc[str(trig),'%'] < 0.9:
        df.loc[name,'Type'] = 'A'
        df.loc[name,'BATSE Type'] = 'A'
    elif trig in mismatch.index:
        df.loc[name,'Type'] = 'D'
        df.loc[name,'BATSE Type'] = 'D'
    df.loc[name,'BATSE Prob.'] = round(abs(int(t=='L') - stability.loc[str(trig),'%']),2)
df

In [ ]:
classification = pd.read_csv('DataFrames/classification_Swift.txt', index_col=0)
stability = pd.read_pickle('DataFrames/stability_Swift.dat')
mismatch = []
for trig in classification.index:
    t = classification.loc[trig,'type']
    #df.loc[trig,'Type'] = t
    df.loc[trig,'Swift Type'] = t
    if stability.loc[str(trig),'%'] < 0.9:
        #df.loc[trig,'Type'] = 'A'
        df.loc[trig,'Swift Type'] = 'A'
    elif trig == 'GRB090813':
        #df.loc[trig,'Type'] = 'D'
        df.loc[trig,'Swift Type'] = 'D'
    elif trig in ['GRB050724','GRB121226A','GRB180418A']:
        #df.loc[trig,'Type'] = 'D'
        df.loc[trig,'Swift Type'] = 'D'
    df.loc[trig,'Swift Prob.'] = round(abs(int(t=='L') - stability.loc[str(trig),'%']),2)
df

In [ ]:
classification = pd.read_csv('DataFrames/classification_Fermi.txt', index_col=0)
stability = pd.read_pickle('DataFrames/stability_Fermi.dat')
mismatch = []
for trig in classification.index:
    index = df.index.tolist()[pd.Index(df.loc[:,'Other Name']).get_loc(trig)]
    t = classification.loc[trig,'type']
    #df.loc[index,'Type'] = t
    df.loc[index,'Fermi Type'] = t
    if stability.loc[str(trig),'%'] < 0.9:
        #df.loc[index,'Type'] = 'A'
        df.loc[index,'Fermi Type'] = 'A'
    elif trig in ['GRB080828189','GRB090811696','GRB110719825','GRB110728056']:
        #df.loc[index,'Type'] = 'D'
        df.loc[index,'Fermi Type'] = 'D'
    df.loc[index,'Fermi Prob.'] = round(abs(int(t=='L') - stability.loc[str(trig),'%']),2)
df

In [ ]:
df.loc[df.Type=='','Type'] = np.where((df['BATSE Type'] == '') & ((df['Swift Type'] == '') | (df['Fermi Type'] == '')),(df['Swift Type'] + df['Fermi Type']),'')[df.Type=='']
sub = df.loc[(df.loc[:,'Swift Type'] != '') & (df.loc[:,'Fermi Type'] != '')]
sub['Type'] = np.where((sub['Swift Type'] == sub['Fermi Type']), sub['Swift Type'], '')
sub.loc[sub.Type=='','Type'] = np.where((sub['Swift Type'] == 'L') & (sub['Fermi Type'] == 'S'), 'O', '')[sub.Type=='']
sub.loc[sub.Type=='','Type'] = np.where((sub['Swift Type'] != 'A'), sub['Swift Type'], sub['Fermi Type'])[sub.Type=='']
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(sub)
df.loc[sub.index,'Type'] = sub['Type']

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    #display(df)
    display(pd.read_pickle('DataFrames/complete_catalog.dat'))

In [ ]:
#df.to_pickle('DataFrames/complete_catalog.dat')
df.to_csv('DataFrames/complete_catalog.txt')

In [ ]:
swift_burst = 'GRB170318B'
fermi_burst = pd.read_csv('DataFrames/intersection_Swift_Fermi.csv',index_col=0).loc[swift_burst]
print(pd.read_pickle('DataFrames/duration_data.dat').loc[swift_burst])
print()
print(pd.read_pickle('DataFrames/duration_data_Fermi.dat').loc[fermi_burst])

In [ ]:
print(df.reset_index().to_latex(columns = ['Name', 'Other Name', 'Type', 'BATSE Type', 'BATSE Prob.', 'Swift Type', 'Swift Prob.', 'Fermi Type', 'Fermi Prob.'], index=False))

In [ ]:
df = pd.read_csv('DataFrames/classification_Swift.txt', index_col=0)

In [ ]:
table = pd.read_table('Downloads/Summary_table.txt', sep = '\s+', skiprows=[0,2], index_col='#', usecols = [0,1])
table = table.loc[pd.read_csv('DataFrames/classification_Swift.txt', index_col=0).index]

In [ ]:
pd.read_pickle('DataFrames/duration_data_Fermi.dat').loc['GRB130925173']

In [ ]:
intersection = pd.read_csv('DataFrames/intersection_Swift_Fermi.csv').iloc[::-1]
Swift = pd.DataFrame(pd.read_pickle('DataFrames/duration_data.dat').loc[:,'T90']).reset_index().iloc[::-1]
Swift = Swift[Swift.isin(intersection.Swift.tolist()).GRBname].reset_index(drop=True)
for i in range(Swift.shape[0]):
    if Swift.iloc[i,1] > 2:
        Swift.iloc[i,1] = 'L ' + str(Swift.iloc[i,1])
    else:
        Swift.iloc[i,1] = 'S ' + str(Swift.iloc[i,1])
display(Swift)
Fermi = pd.DataFrame(pd.read_pickle('DataFrames/duration_data_Fermi.dat').loc[:,'T90']).reset_index()
Fermi = Fermi[Fermi.isin(intersection.Fermi.tolist()).name].reset_index(drop=True)
for i in range(Fermi.shape[0]):
    if Fermi.iloc[i,1] > 2:
        Fermi.iloc[i,1] = 'L ' + str(Fermi.iloc[i,1])
    else:
        Fermi.iloc[i,1] = 'S ' + str(Fermi.iloc[i,1])
display(Fermi)
classification = pd.concat([Swift,Fermi], axis=1)
classification = classification[classification.iloc[:,1].str[0] != classification.iloc[:,3].str[0]].reset_index(drop=True)
classification.columns = ['Swift', 'type', 'Fermi', 'type']
display(classification)
#classification.set_index('Swift').to_csv('DataFrames/duration_disagreement.csv')

In [ ]:
#pd.read_pickle('DataFrames/complete_catalog.dat').to_csv('DataFrames/complete_catalog.csv')